In [ ]:
import girder_client 
%load_ext autoreload
%autoreload 2
#import adrcSchemaHelpers as hlprs
import metadata_pipeline as mph 
import brain_region_maps as brm

#gc = girder_client.GirderClient(apiUrl="https://styx.neurology.emory.edu/girder/api/v1")
gc = girder_client.GirderClient(apiUrl="http://candygram.neurology.emory.edu:8080/api/v1")
_ = gc.authenticate(interactive=True)

In [ ]:
NeuroPathDemoImages_Collection='638e2da11f75016b81fda12f'

In [ ]:
def getPtSlideDataSummary( slideSetMetaData,debug=True):
    ## Expects an array of all of the items below a folder, so for exampel  all of the slides for case E20-73
    ## This should then return the number of control slides found, number of files that are missing metadata
    ## completely, and also invalid metadata
    ctrlSlides = 0
    validatedSlides = 0
    notValid = 0
    slidesToReview = []
    
    for i in slideSetMetaData:
        if 'npSchema' not in i['meta']:
            notValid+=1
            slidesToReview.append(i)
        elif ('controlSlide' in i['meta']['npSchema']):
            ctrlSlides+=1
        else:
            validItem,meta = mph.validateMetadata(i['meta'])
            ### Now process errors from 
            if not validItem:
                notValid+=1
                slidesToReview.append(i)
            else:
                validatedSlides+=1
    if debug:
        pass
        #print("Valid:%d,Controls:%d,NotValid:%d" % (validatedSlides,ctrlSlides,notValid))
        
        
    return {"status":"parsed", "valid":validatedSlides,"controlSlides": ctrlSlides, "notValid": notValid, "slidesToReview":slidesToReview}

In [ ]:
# testOutput = mph.populateMetadata(gc, folderID='638e2dea1f75016b81fda132', outputFailed=True)


In [ ]:
## Need to do this on a per case basis
needsBrainRegionMap = []
slideSummaryData = {}

for yrf in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
    year = yrf['name']
    if year.isdigit():
        slideSummaryData[year] = {} ## Going to create a stats dictionary ordered by year/pt
        
        for pt in gc.listFolder(yrf['_id']):
            slideSummaryData[year][pt['name']] = {}
            
            if pt['name'] not in brm.MAP:
                #print(year,pt['name'],"Needs Brain Region Map")
                slideSummaryData[year][pt['name']] = {"status": "incomplete", "reason": "needs brain region map"}
        
            else:

                ptItemData = mph.getFolderContents(gc,pt['_id']) ### Get all of the items cfor selected year
        
                stats = getPtSlideDataSummary(ptItemData,debug=True)
                slideSummaryData[year][pt['name']] = stats

In [ ]:
needsMap = []
for yr in slideSummaryData:
    for pt in slideSummaryData[yr]:
        caseMeta = slideSummaryData[yr][pt]
        if caseMeta['status'] == 'parsed':
            print(yr,pt,caseMeta['valid'],caseMeta['notValid'])
        elif caseMeta['status'] == 'incomplete':
            needsMap.append((yr,pt))

In [ ]:
needsMap

In [ ]:
mph.auditMetadata(gc,collectionID=NeuroPathDemoImages_Collection,outputRecords=True)

In [ ]:
# folder2020 = '638e81331f75016b81febf39'

# itemData2020 = mph.getFolderContents(gc,folder2020)

In [ ]:
allNpWorking = []

for itm in itemData2020:
    if 'meta' in itm and 'npWorking' in itm['meta']:
        allNpWorking.append(itm)

In [ ]:
[(x['meta']['npWorking'],x['name']) for x in allNpWorking]

In [ ]:
for pt in slideSummaryData['2017'].keys():
    s=  slideSummaryData['2017'][pt]
    if 'valid' in s:
        print('2017',pt,s['valid'],s['notValid'],s['controlSlides'])
    

In [ ]:
for i in slideSummaryData['2017']['E17-39']['slidesToReview']:
    print(i)

In [ ]:
allItemData = mph.get

In [ ]:
# for y in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     if y['name'].startswith('20'):
#         for ptId in gc.listFolder(y['_id']):
#             print(ptId['name'])
#             hlprs.scanMetadata(gc,ptId,updateGirder=True)

In [ ]:
# ## Now also flatten the views so in the image organizer app I don't have to upload

# for y in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     if y['name'].startswith('20'):
#         for ptId in gc.listFolder(y['_id']):
#             gc.addMetadataToFolder(ptId['_id'],{"isLinear":True, "caseViewFlag":1})

In [ ]:
# ### Get a list of every patient ID... this is quick, it's just from the main folder name!!
# ptIdList = []

# for yrf in gc.listFolder(NeuroPathDemoImages_Collection,parentFolderType='collection'):
#     year = yrf['name']
#     if year.isdigit():
#         for ptf in gc.listFolder(yrf['_id']):
# #             print("\t",ptf['name'])
#             ptIdList.append(ptf['name'])